# Batch pre-computed recommendations

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import sys

import redis
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel
from tqdm.auto import tqdm

load_dotenv()

sys.path.insert(0, "..")

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    run_name: str = "005-refactor"
    notebook_persist_dp: str = None
    random_seed: int = 41

    redis_host: str = "localhost"
    redis_port: int = 6379
    redis_key_prefix: str = "output:i2i:"

    batch_recs_fp: str = "data/003-alias-champion/batch_recs.jsonl"

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        os.makedirs(self.notebook_persist_dp, exist_ok=True)
        self.batch_recs_fp = f"{self.notebook_persist_dp}/batch_recs.jsonl"

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

{
  "testing": false,
  "run_name": "005-refactor",
  "notebook_persist_dp": "/Users/dvq/frostmourne/fsds/fsds-recsys/chapters/l7/notebooks/data/005-refactor",
  "random_seed": 41,
  "redis_host": "localhost",
  "redis_port": 6379,
  "redis_key_prefix": "output:i2i:",
  "batch_recs_fp": "/Users/dvq/frostmourne/fsds/fsds-recsys/chapters/l7/notebooks/data/005-refactor/batch_recs.jsonl"
}


# Load batch recs into Redis

In [4]:
r = redis.Redis(host=args.redis_host, port=args.redis_port, db=0, decode_responses=True)
assert (
    r.ping()
), f"Redis at {args.redis_host}:{args.port} is not running, please make sure you have started the Redis docker service"

In [5]:
def store_recommendations(file_path: str):
    with open(file_path, "r") as file:
        for line in tqdm(file):
            rec_data = json.loads(line)
            target_item = rec_data["target_item"]
            key = args.redis_key_prefix + target_item
            r.set(
                key,
                json.dumps(
                    {
                        "rec_item_ids": rec_data["rec_item_ids"],
                        "rec_scores": rec_data["rec_scores"],
                    }
                ),
            )


def get_recommendations(target_item):
    key = args.redis_key_prefix + target_item
    rec_data = r.get(target_item)
    if rec_data:
        return json.loads(rec_data)
    return None


def get_example_keys(count=5):
    keys = r.scan_iter(match=args.redis_key_prefix + "*", count=count)
    output = []
    for i, key in enumerate(keys, 1):
        output.append(key)
        if i >= count:
            return output

In [6]:
logger.info(f"Loading batch recs output from {args.batch_recs_fp}...")
store_recommendations(args.batch_recs_fp)

2024-10-07 21:20:45.063 | INFO     | __main__:<module>:1 - Loading batch recs output from /Users/dvq/frostmourne/fsds/fsds-recsys/chapters/l7/notebooks/data/005-refactor/batch_recs.jsonl...


0it [00:00, ?it/s]

In [7]:
get_recommendations(get_example_keys()[0])

{'rec_item_ids': ['B00DUARDFC',
  'B00PZ81VVE',
  'B00LSFRY3C',
  'B0147ET0ZA',
  'B00A7VK16A',
  'B00NFXOQW2',
  'B001PIY49O',
  'B01MT1DCIV',
  'B00A8306C6',
  'B00HS5FETM',
  'B015PXH9S2',
  'B00HRKE1SI',
  'B011L4D3BG',
  'B00IPTUJ8G',
  'B00OBZNI0O',
  'B0C37RBK2R',
  'B01LW6SS1X',
  'B001I75UIQ',
  'B01C48EA1A',
  'B00J5C3SXK',
  'B00QKLN4OW',
  'B00H89IBFM',
  'B079ZPD1BV',
  'B00COAGPG2',
  'B00TL7LUHA',
  'B002W8SBIG',
  'B00B1N07QM',
  'B07YNHSV65',
  'B00VMB5VCS',
  'B00T8F5VKW',
  'B073SRS2RW',
  'B077W2Z2SF',
  'B073ZR63P8',
  'B00FATRKWM',
  'B089GHV1C7',
  'B00UBL0ZQC',
  'B071FSRJXV',
  'B005EOOTHW',
  'B00AIALGZK',
  'B01D30Q868',
  'B0BQGYB74T',
  'B00NET618O',
  'B09K3ZY6BR',
  'B07PDSWRTD',
  'B00PGLG79G',
  'B002I0JNFI',
  'B0001Q5SG4',
  'B01GW8ZC2O',
  'B0794W1LWG',
  'B0163H5F30',
  'B0BJ6VST99',
  'B07B416X7V',
  'B0BF1BQ3D2',
  'B01KX8ETK0',
  'B07DGSD6FZ',
  'B000HAWCFE',
  'B006VR65D2',
  'B000X37732',
  'B00ZKZY04M',
  'B07JW5HRN7',
  'B071YZVS75',
  'B001E